In [2]:
import cv2, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_openml


X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
X = (X / 255.0).astype(np.float32)
y = y.astype(np.uint8)
clf = LogisticRegression(
    solver="lbfgs",
    multi_class="multinomial",
    max_iter=1500,
    n_jobs=-1,
    C=2.0,
    random_state=42,
).fit(X, y)


def binarize(gray):
    """Otsu threshold; makes digits white on black."""
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    
    _, inv = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    _, dir = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    r_inv, r_dir = np.mean(inv == 255), np.mean(dir == 255)
    m = inv if 0.02 < r_inv < 0.6 else dir  # رقم‌ها سفید باشند و نه خیلی کم/زیاد
    # کمی بستن و دایلیشن سبک برای وصل شدن خطوط باریک (مثل 4)
    k = np.ones((3, 3), np.uint8)
    m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, k, iterations=1)
    m = cv2.dilate(m, k, iterations=1)
    return m

def safe_box(x, y, w, h, pad, W, H):
    x1 = max(0, x - pad); y1 = max(0, y - pad)
    x2 = min(W, x + w + pad); y2 = min(H, y + h + pad)
    return x1, y1, x2, y2

def to_mnist(crop_mask):
    
    
    if np.mean(crop_mask > 0) > 0.5:
        crop_mask = 255 - crop_mask

    ys, xs = np.where(crop_mask > 0)
    if ys.size == 0: 
        return None

    y1, y2 = ys.min(), ys.max() + 1
    x1, x2 = xs.min(), xs.max() + 1
    digit = crop_mask[y1:y2, x1:x2]

    h, w = digit.shape
    if h == 0 or w == 0:
        return None

    if h > w:
        nh, nw = 20, max(1, int(round(20.0 * w / h)))
    else:
        nw, nh = 20, max(1, int(round(20.0 * h / w)))
    digit = cv2.resize(digit, (nw, nh), interpolation=cv2.INTER_AREA)

    canvas = np.zeros((28, 28), np.uint8)
    xoff = (28 - nw) // 2
    yoff = (28 - nh) // 2
    canvas[yoff:yoff + nh, xoff:xoff + nw] = digit

    M = cv2.moments(canvas)
    if M["m00"] != 0:
        cx, cy = M["m10"] / M["m00"], M["m01"] / M["m00"]
        dx, dy = int(round(14 - cx)), int(round(14 - cy))
        canvas = cv2.warpAffine(canvas, np.float32([[1, 0, dx], [0, 1, dy]]), (28, 28),
                                flags=cv2.INTER_LINEAR, borderValue=0)

    return (canvas.astype(np.float32) / 255.0).reshape(1, -1)



img = cv2.imread("sample.png")
if img is None:
    raise FileNotFoundError("sample.png not found")

H, W = img.shape[:2]
vis = img.copy()  

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
mask = binarize(gray)


cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

min_area = max(120, int(0.0002 * H * W))
boxes = [cv2.boundingRect(c) for c in cnts if cv2.contourArea(c) >= min_area]


row_h = max(20, int(0.06 * H))
boxes = sorted(boxes, key=lambda b: (b[1] // row_h, b[0]))

for (x, y, w, h) in boxes:
    x1, y1, x2, y2 = safe_box(x, y, w, h, pad=10, W=W, H=H)
    crop_mask = mask[y1:y2, x1:x2]
    feat = to_mnist(crop_mask)
    if feat is None:
        continue

    pred = int(clf.predict(feat)[0])

    
    cv2.rectangle(vis, (x1, y1), (x2, y2), (0, 255, 0), 2)  # سبز
    ty = y1 - 8
    if ty < 16:  # اگر جا نبود، بره زیر کادر
        ty = min(y2 + 20, H - 5)
    cv2.putText(vis, str(pred), (x1, ty),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2, cv2.LINE_AA)  # آبی

cv2.imwrite("labeled_sample.png", vis)
print("saved -> labeled_sample.png")


C:\Users\Hani\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


saved -> labeled_sample.png
